In [1]:
import requests
# Get all the games that the player played
def getownedgames(idnum):
    """Takes the id number of a steam user and returns the id num of games they own"""
    url='http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3A629E52AE594CB777253431189F2967&steamid={}&format=json'.format(idnum)
    r=requests.get(url)
    json_data=r.json()
    return json_data

In [2]:
def get_reviews(gameid,num):
    """Takes the id number of a game and returns 20 reviews"""
    url='http://store.steampowered.com/appreviews/{}?json=1&start_offset={} '.format(gameid,num)
    r=requests.get(url)
    json_data=r.json()
    return json_data

In [3]:
import pandas as pd
def get_all_reviews(gameid):
    """Uses 'get_review' function to pull as many reviews as it can from steam website"""
    """STEAM website usually have limitation (20 reviews) on how many reviews we can pull"""
    n=0
    stop=0
    fst_iterat=get_reviews(gameid,n)
    if fst_iterat['success']==1:
        df=pd.DataFrame(fst_iterat['reviews'])
        numreview=fst_iterat['query_summary']['num_reviews']             
        while numreview==20:
            if len(df)>=120:
                break
            n+=20
            add_iterat=get_reviews(gameid,n)
            df_add=pd.DataFrame(add_iterat['reviews'])
            df=df.append(df_add)
            numreview=add_iterat['query_summary']['num_reviews']
        df_author=pd.DataFrame(df['author'].tolist())
        df['author']=df_author['steamid']
    return df

Get all the application id that is on STEAM website.

In [4]:
api_key='3A629E52AE594CB777253431189F2967' # API key for getting information from Steam
allgame='http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key={}&format=json'.format(api_key)
all_r=requests.get(allgame)
allgame_id=all_r.json()

In [5]:
steamapp_dict=dict()
for num in range(len(allgame_id['applist']['apps'])):
    key=allgame_id['applist']['apps'][num]['appid']
    value=allgame_id['applist']['apps'][num]['name']
    if key in steamapp_dict:
        continue
    else:
        steamapp_dict[key]=value

In [7]:
import pickle

In [19]:
file_name='testfile'
review_df=pickle.load(open(file_name,"rb"))

In [8]:
file_name='got_review_14000.p'
review_df=pickle.load(open(file_name,"rb"))

In [13]:
error=list()
recommended_games=list()
for name in review_df['game_name']:
    try:
        ids=list(steamapp_dict.keys())[list(steamapp_dict.values()).index(name)]
        recommended_games.append(ids)
    except:
        error.append(name)

In [14]:
error

['Dream Beach - Mallorca | VR Relaxation Sphaere | 360° Video | 6K/2D',
 'Attracting Abundance | VR Motivation Sphaere | 360° Video | 6K/2D',
 'Sun & Clouds Timelapse | VR Travel Sphaere | 360° Video | 6K/2D',
 'Pebbly Beach Nature Meditation | VR Meditation Sphaere | 360° Video | 6K/2D']

In [17]:
prev_ids=recommended_games

In [18]:
# get all keys
allkeys=list(steamapp_dict.keys())
# remove keys included in previous pickle
newkeys = [key for key in allkeys if key not in prev_ids]

In [ ]:
import pickle
i=len(prev_ids)
for key in newkeys:
    try:
        df=get_all_reviews(key)
    except: 
        continue
        print('failed')
    df['game_name']=steamapp_dict.get(key)
    if 'review_df' not in locals():
        review_df=df
    else:
        review_df=review_df.append(df, ignore_index=True, sort=False)
        i=i+1
        if i%1000==0:
            print(i)
            name='got_review_{}.p'.format(i)
            pickle.dump(review_df,open(name,"wb"))

108000
109000


C:\Users\shshi\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


110000
111000
112000
113000


In [ ]:
name

In [ ]:
pickle.dump(review_df,open("final.p","wb"))